In [49]:
import jax
import jax.numpy as jnp
import os; os.chdir("..")

In [50]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import jax.numpy as jnp
from benchmarks.simple_QP.load_simple_QP import (
    SimpleQPDataset,
    create_dataloaders,
    dc3_dataloader,
)

dataset_dir = "/home/aterpin/projects/hcnn/src/benchmarks/simple_QP/datasets"

def load_data(
    use_DC3_dataset,
    use_convex,
    problem_seed,
    problem_var,
    problem_nineq,
    problem_neq,
    problem_examples,
):
    """Load problem data."""
    if not use_DC3_dataset:
        # Choose problem parameters
        if use_convex:
            filename = (
                f"SimpleQP_seed{problem_seed}_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_examples{problem_examples}.npz"
            )
        else:
            raise NotImplementedError()
        dataset_path = os.path.join(dataset_dir, filename)

        QPDataset = SimpleQPDataset(dataset_path)
        train_loader, valid_loader, test_loader = create_dataloaders(
            dataset_path, batch_size=2048, val_split=0.1, test_split=0.1
        )
        Q, p, A, G, h = QPDataset.const
        p = p[0, :, :]
        X = QPDataset.X
    else:
        # Choose the filename here
        if use_convex:
            filename = (
                f"dc3_random_simple_dataset_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_ex{problem_examples}"
            )
        else:
            filename = (
                f"dc3_random_nonconvex_dataset_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_ex{problem_examples}"
            )
        filename_train = filename + "train.npz"
        dataset_path_train = os.path.join(dataset_dir, filename_train)
        filename_valid = filename + "valid.npz"
        dataset_path_valid = os.path.join(dataset_dir, filename_valid)
        filename_test = filename + "test.npz"
        dataset_path_test = os.path.join(dataset_dir, filename_test)
        train_loader = dc3_dataloader(dataset_path_train, use_convex, batch_size=2048)
        valid_loader = dc3_dataloader(
            dataset_path_valid, use_convex, batch_size=1024, shuffle=False
        )
        test_loader = dc3_dataloader(
            dataset_path_test, use_convex, batch_size=1024, shuffle=False
        )
        Q, p, A, G, h = train_loader.dataset.const
        p = p[0, :, :]
        X = train_loader.dataset.X

    return (filename, Q, p, A, G, h, X, train_loader, valid_loader, test_loader)

In [51]:
# 1. Load a batch of data for autotuning
filename, Q, p, A, G, h, X, train_loader, valid_loader, test_loader = load_data(
    use_DC3_dataset=True,
    use_convex=True,
    problem_seed=42,
    problem_var=100,
    problem_nineq=50,
    problem_neq=50,
    problem_examples=10000,
)
X_batch, _ = next(iter(train_loader))

In [5]:
import signal
class GracefulShutdown:
    """A context manager for graceful shutdowns."""

    stop = False

    def __enter__(self):
        """Register the signal handler."""

        def handle_signal(signum, frame):
            self.stop = True

        signal.signal(signal.SIGINT, handle_signal)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Unregister the signal handler."""
        pass

In [54]:
def get_n_iter_backward(n_iter_train):
    if n_iter_train <= 25:
        return n_iter_train
    elif n_iter_train <= 250:
        return 25
    return 25 + (n_iter_train - 250) // 10

def build_evaluate_params(x, n_iter, project, compute_cv):
    def evaluate_params(init, sigma):
        y, init = project(init=init, x=x, sigma=sigma, n_iter=n_iter)
        cvs = compute_cv(y)
        values = jnp.linalg.norm(y - x)
        return init, jnp.max(cvs), jnp.mean(values)
    return evaluate_params

In [55]:
# TODO: replace with the actual project and compute_cv functions
project = lambda init, x, sigma, n_iter: (x, init)
compute_cv = lambda y: jnp.zeros((y.shape[0],))
x = jnp.zeros((128, 1)) # batch of the points to project

In [ ]:
target_cv = 10
target_rs = 4
n_max_iter = 1000
n_iter_step = 100
n_iter_candidates = n_max_iter // n_iter_step
tie_breaker = "cv"
if tie_breaker == "cv":
    id_tie_breaker = 0
elif tie_breaker == "rs":
    id_tie_breaker = 1

sigma_candidates = jnp.arange(0.0, 10.05, 0.05)
sigma_candidates = sigma_candidates.at[0].set(0.01)

to_project_shape = (128, 100, 1)

eval_fn = jax.jit(build_evaluate_params(x, n_iter_step, project, compute_cv))

results = jnp.inf * jnp.ones((len(sigma_candidates), n_iter_candidates, 2))
def body_fun(i, r):
    sigma = sigma_candidates[i]
    def body_fun_i(j, state_i):
        ri, init = state_i
        init, cv, val = eval_fn(init, sigma)
        return ri.at[j, :].set(jnp.stack([cv, val])), init
    init = jnp.zeros(to_project_shape)
    _r, _ = jax.lax.fori_loop(0, n_iter_candidates, body_fun, (r[i, ...], init))
    return r.at[i, ...].set(_r)
results = jax.lax.fori_loop(0, len(sigma_candidates), body_fun, results)

TypeError: tuple indices must be integers or slices, not tuple

In [48]:
# Use the best result as proxy for the optimal value
opt = jnp.min(results[:, :, 1])
# Compute the relative suboptimality
rs = (results[:, :, 1] - opt) / (opt + 1e-20)
# Compute which entries satisfy both target conditions
mask = (results[:, :, 0] < target_cv) * (rs < target_rs)
mask_valid_sigma = jnp.any(mask, axis=1)
if jnp.sum(mask) == 0:
    raise ValueError("No valid sigma found for the given target conditions.")
# For each row, find the first column index where the condition is met
first_valid_idx = jnp.argmax(mask, axis=1)
# Find the minimum number of iterations across all rows
min_iter_idx = jnp.min(first_valid_idx[mask_valid_sigma])
# Find the best sigma values
best_sigma_mask = (first_valid_idx == min_iter_idx) * mask_valid_sigma

if jnp.sum(best_sigma_mask) > 1:
    # Tie breaking
    mask = mask * best_sigma_mask[:, None]
    min_val = jnp.min(results[mask, id_tie_breaker])
    best_sigma_mask = best_sigma_mask & jnp.any(
        results[..., id_tie_breaker] == min_val, axis=1)
    if jnp.sum(best_sigma_mask) > 1:
        # Other tie breaking
        mask = mask * best_sigma_mask[:, None]
        min_val = jnp.min(results[mask, 1 - id_tie_breaker])
        best_sigma_mask = best_sigma_mask & jnp.any(
            results[..., id_tie_breaker] == min_val, axis=1)
        
# Find the (first) index non-zero in best_sigma_mask
best_sigma_idx = jnp.argmax(best_sigma_mask)
best_sigma = sigma_candidates[best_sigma_idx]
best_n_iter = n_iter_step * (min_iter_idx + 1)
